In [1]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, GenerationConfig
import torch
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory,ConversationSummaryMemory, ConversationBufferWindowMemory
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser, CommaSeparatedListOutputParser

In [2]:
model_id = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
pipe = pipeline("text-generation", 
                model=model, 
                tokenizer=tokenizer,
                device=0, 
                max_new_tokens=2000,
                do_sample=True, 
                top_k=20, 
                top_p=0.7,
                early_stopping=True,
                num_beams=2
               )
hf = HuggingFacePipeline(pipeline=pipe)

In [5]:
question_schema = ResponseSchema(name='Question', description="Question Generated on the given topic")
choice1_schema = ResponseSchema(name='Choice1', description='Choice 1 for the given question')
choice2_schema = ResponseSchema(name='Choice2', description='Choice 2 for the given question')
choice3_schema = ResponseSchema(name='Choice3', description='Choice 3 for the given question')
choice4_schema = ResponseSchema(name='Choice4', description='Choice 4 for the given question')
answer_schema = ResponseSchema(name='Answer', description='One of the selected choices out of 4 choices given as the answer. Eg Choice1')
explanation_schema = ResponseSchema(name='Explanation', description = 'Explanation why a particular choice is selected as the answer')

response_schemas = [question_schema, 
                    choice1_schema,
                    choice2_schema,
                    choice3_schema,
                    choice4_schema,
                    answer_schema,
                    explanation_schema
                   ]

In [6]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
# focused on the topic of {subject}
#{chat_history}

In [183]:
# template="""You are a KnowledgeStore tasked with generating a single MCQ type question for graduate students of a specified difficulty level focused on a particular subject. This question should present a significant challenge, incorporating real-world applications or data to contextualize mathematical or statistical concepts involved. It requires a deep understanding and application of theoretical principles, possibly involving multiple steps or the integration of several concepts.

# Previously Generated Questions.
# {chat_history}

# Human:
# It is crucial that this task results in exactly one multiple-choice question with four choices: Choice1, Choice2, Choice3, Choice4. The correct answer must be one of these choices. Despite any challenges in meeting all specified requirements, the generation of more than one question is not acceptable.
# Ensure the question is of {difficulty} difficulty level, centered around {subject}.
# Please generate a single question even if it is not possible to meet all the above criteria perfectly. The priority is to adhere to the format and quantity specified: one question, four choices.

# {format_instructions}
# """


In [208]:
# template="""You are a Quiz Master expert in generating a Single MCQ Question on a given subject and difficulty level.
# Previously Generated Questions.
# {chat_history}

# Human: Please generate a single MCQ questions with the details mentioned below:
# - Subject: {subject}
# - Difficulty : {difficulty}
# - Number of Choices: 4 [Choice 1, Choice2, Choice3, Choice4]
# - The answer should be among the 4 choices only.
# - Only a single MCQ Question is expected. More than 1 question is not acceptable.

# {format_instructions}
# """

In [238]:
# template = """You are a KnowledgeStore which generates MCQ questions of desired difficulty on a given topic.
# You can generate really good quality questions which are different from the questions generated previously.

# Previously Generated Questions.
# {chat_history}

# Human: Generate a single MCQ type question for graduate students of {difficulty} difficulty level focussed on {subject}. 
# The question should present a significant challenge, and should ideally incorporate real-world applications or data to contextualize the mathematical or statistical concepts involved. 
# Ensure the question demand a deep understanding and application of theoretical principles, possibly involving multiple steps or the integration of several concepts.
# Ensure that Question is strictly multiple choice question with exactly 4 choices. Choice1, Choice2, Choice3, Choice4. The answer should definitely be one of the Choices.
# You can ignore other conditions but it is compulsory to be a single question.

# {format_instructions}
# """

In [335]:
template = """You are a KnowledgeStore which generates MCQ questions of desired difficulty on a given topic.
You can generate really good quality questions which are different from the questions generated previously.

Previously Generated Questions.
{chat_history}

Human: Generate a single MCQ type question of {difficulty} difficulty level focussed on {subject}.
The question should be totally different from previous generated quesions mentioned above.
The question should present a significant challenge, and should ideally incorporate real-world applications or data to contextualize the mathematical or statistical concepts involved. 
Ensure the question demand a deep understanding and application of theoretical principles, possibly involving multiple steps or the integration of several concepts.
Ensure that Question is strictly multiple choice question with exactly 4 choices. 
Choice1, Choice2, Choice3, Choice4. The answer should definitely be one of the Choices.
It is essential to make sure you generate a single question even if you are not able to meet all the requirments.
You can ignore other requirements if not met but it is compulsory to be a single MCQ question with 4 Choices.

{format_instructions}
Output:
"""

#
prompt = PromptTemplate.from_template(template)
prompt = PromptTemplate(
    input_variables=["chat_history", 
                     "difficulty",
                     "subject",
                     "format_instructions"], template=template
)
#memory = ConversationSummaryMemory(llm=hf, input_key='subject')
#memory = ConversationBufferWindowMemory(k=5, memory_key="chat_history", input_key='subject')
#memory = ConversationBufferMemory(memory_key="chat_history", input_key='subject')

In [380]:
print(template)

You are a KnowledgeStore which generates MCQ questions of desired difficulty on a given topic.
You can generate really good quality questions which are different from the questions generated previously.

Previously Generated Questions.
{chat_history}

Human: Generate a single MCQ type question of {difficulty} difficulty level focussed on {subject}.
The question should be totally different from previous generated quesions mentioned above.
The question should present a significant challenge, and should ideally incorporate real-world applications or data to contextualize the mathematical or statistical concepts involved. 
Ensure the question demand a deep understanding and application of theoretical principles, possibly involving multiple steps or the integration of several concepts.
Ensure that Question is strictly multiple choice question with exactly 4 choices. 
Choice1, Choice2, Choice3, Choice4. The answer should definitely be one of the Choices.
It is essential to make sure you gene

In [311]:
#memory.load_memory_variables({})

In [336]:
llm_chain = LLMChain(
    llm=hf,
    prompt=prompt,
    #memory=memory,
    verbose=True
)

In [337]:
chat_history = ""

In [376]:
out = llm_chain.run({'chat_history' : chat_history,
                    'difficulty' : 'easy',
                     'subject' : 'history of USA',
                     'format_instructions' : format_instructions
                    })

/home/aagarwal/miniconda3/envs/OpenShape/lib/python3.9/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new LLMChain chain...
Prompt after formatting:
You are a KnowledgeStore which generates MCQ questions of desired difficulty on a given topic.
You can generate really good quality questions which are different from the questions generated previously.

Previously Generated Questions.
Which event marked the beginning of the American Revolution?
Which of these U.S. Presidents was born in a log cabin?
Which U.S. President signed the Emancipation Proclamation, freeing all slaves in Confederate territory?
Which U.S. President was the first to be impeached by the House of Representatives?
Which U.S. President is known for his 'New Deal' policies that helped the country recover from the Great Depression?
Which U.S. President was the first to establish a national forest?
Which U.S. President was the first to establish a national monument dedicated to peace?
Which U.S. President was the first to establish a national park dedicated to a U.S. President?


Human: Generate a single MCQ t

In [377]:
print(out)

```json
{
	"Question": "Which U.S. President was the first to establish a national holiday dedicated to labor?",
	"Choice1": "George Washington",
	"Choice2": "Thomas Jefferson",
	"Choice3": "Abraham Lincoln",
	"Choice4": "Franklin D. Roosevelt",
	"Answer": "Franklin D. Roosevelt",
	"Explanation": "Franklin D. Roosevelt established Labor Day as a national holiday in 1935."
}
```


In [378]:
question = output_parser.parse(out).get('Question')
question

'Which U.S. President was the first to establish a national holiday dedicated to labor?'

In [379]:
chat_history+=f"{question}\n"
print(chat_history)

Which event marked the beginning of the American Revolution?
Which of these U.S. Presidents was born in a log cabin?
Which U.S. President signed the Emancipation Proclamation, freeing all slaves in Confederate territory?
Which U.S. President was the first to be impeached by the House of Representatives?
Which U.S. President is known for his 'New Deal' policies that helped the country recover from the Great Depression?
Which U.S. President was the first to establish a national forest?
Which U.S. President was the first to establish a national monument dedicated to peace?
Which U.S. President was the first to establish a national park dedicated to a U.S. President?
Which U.S. President was the first to establish a national holiday dedicated to labor?

